# Financial App

In [1]:
import os
import pandas as pd

# Change working directory to the project root

# Get the current working directory
os.chdir("..")
# Verify the current working directory
print(f"Now in directory: {os.getcwd()}")

# Import your module
from app.modules.database.connection import engine
from app.modules.database.credit_manager import new_credit

%run logs/run_log.py

Now in directory: c:\Users\juanm_8qa8lav\Documents\Proyectos_Personales\FinancialApp
Nuevo plan de negocio creado, su id es 1 y fue asociado a la empresa NeoCrediT S.A.
Nuevo plan de negocio creado, su id es 2 y fue asociado a la empresa Onoyen S.R.L.
Nuevo plan de negocio creado, su id es 3 y fue asociado a la empresa Asociación Mutual Union Federal
Nuevo plan de negocio creado, su id es 4 y fue asociado a la empresa Asociación Mutual Union Federal


In [2]:
df = pd.read_sql('credits', engine, index_col='ID')

df

,ID_External,ID_Client,Date_Settlement,ID_BP,Cap_Requested,Cap_Grant,N_Inst,First_Inst_Purch,TEM_W_IVA,V_Inst,First_Inst_Sold,D_F_Due,ID_Purch,ID_Sale
ID,,,,,,,,,,,,,,


In [3]:
setts = pd.read_sql('settings', engine, index_col='ID')

setts

,Detail,Type,Value
ID,,,
1,Día de Vto. Predeterminado,I,28
2,Periodos de Gracia,I,2


In [4]:
customers = pd.read_sql('customers', engine, index_col='ID')

customers

,CUIL,DNI,Last_Name,Name,Gender,Date_Birth,Age_at_Discharge,ID_Province,Locality,Street,...,Salary,CBU,Collection_Entity,Employer,Dependence,CUIT_Employer,ID_Empl_Prov,Empl_Loc,Empl_Adress,Last_Update
ID,,,,,,,,,,,,,,,,,,,,,
1,20363297588,36329758,Carini,Juan Martín,M,1992-04-01,32,1,Bahía Blanca,D'Orbigny,...,900000.0,None,Fideisa,Fideisa,Fideisa,30709706736,1,Bahía Blanca,Alem 250,2024-11-29 10:25:17


In [5]:
pd.read_sql('installments', engine, index_col='ID')

,ID_Op,Nro_Inst,D_Due,Capital,Interest,IVA,Total,ID_Owner
ID,,,,,,,,


In [6]:
nc, inst = new_credit(
    id_customer = 1,
    Date_Settlement = pd.Timestamp("2024/11/28"),
    ID_BP = 4,
    Cap_Requested = 10**6,
    Cap_Grant = 10**6,
    N_Inst = 6,
    TEM_W_IVA = 1.88/365*30,
    V_Inst = None)

ID_External                        None
ID_Client                             1
Date_Settlement     2024-11-28 00:00:00
ID_BP                                 4
Cap_Requested                 1000000.0
Cap_Grant                     1000000.0
N_Inst                                6
First_Inst_Purch                      0
TEM_W_IVA                      0.154521
V_Inst                        267461.42
First_Inst_Sold                       0
D_F_Due             2025-01-28 00:00:00
ID_Purch                           None
ID_Sale                            None
Name: 1, dtype: object


c:\Users\juanm_8qa8lav\Documents\Proyectos_Personales\FinancialApp\app\modules\database\credit_manager.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_cr.loc[id, [


In [7]:
df = pd.read_sql('credits', engine, index_col='ID')

df

,ID_External,ID_Client,Date_Settlement,ID_BP,Cap_Requested,Cap_Grant,N_Inst,First_Inst_Purch,TEM_W_IVA,V_Inst,First_Inst_Sold,D_F_Due,ID_Purch,ID_Sale
ID,,,,,,,,,,,,,,
1,None,1,2024-11-28,4,1000000.0,1000000.0,6,0,0.154521,267461.42,0,2025-01-28,None,None


In [8]:
df = pd.read_sql('installments', engine, index_col='ID')
df = df.loc[df['ID_Op'] == 1]

df.loc['Total', ['Capital', 'Interest', 'IVA', 'Total']] = df[['Capital', 'Interest', 'IVA', 'Total']].sum()

print(f'{df.loc['Total', 'Capital'] + df.loc['Total', 'Interest'] + df.loc['Total', 'IVA'] - df.loc['Total', 'Total']:,.2f}')

df[['Capital', 'Interest', 'IVA', 'Total']] = df[['Capital', 'Interest', 'IVA', 'Total']].map('${:,.2f}'.format)
df

-0.02


,ID_Op,Nro_Inst,D_Due,Capital,Interest,IVA,Total,ID_Owner
ID,,,,,,,,
1,1.0,1.0,2025-01-28,"$112,940.88","$127,702.93","$26,817.62","$267,461.43",1.0
2,1.0,2.0,2025-01-28,"$130,392.56","$113,280.05","$23,788.81","$267,461.43",1.0
3,1.0,3.0,2025-02-28,"$150,540.89","$96,628.54","$20,291.99","$267,461.43",1.0
4,1.0,4.0,2025-03-28,"$173,802.55","$77,404.03","$16,254.85","$267,461.43",1.0
5,1.0,5.0,2025-04-28,"$200,658.62","$55,208.93","$11,593.88","$267,461.43",1.0
6,1.0,6.0,2025-05-28,"$231,664.50","$29,584.24","$6,212.69","$267,461.43",1.0
Total,NaN,NaN,NaT,"$1,000,000.00","$499,808.72","$104,959.84","$1,604,768.58",NaN


In [9]:
nc, inst = new_credit(
    id_customer = 1,
    Date_Settlement = pd.Timestamp("2024/11/28"),
    ID_BP = 4,
    Cap_Requested = 0.5*10**6,
    Cap_Grant = 0.5*10**6,
    N_Inst = 12,
    TEM_W_IVA = 1.96/365*30,
    V_Inst = None)

ID_External                        None
ID_Client                             1
Date_Settlement     2024-11-28 00:00:00
ID_BP                                 4
Cap_Requested                  500000.0
Cap_Grant                      500000.0
N_Inst                               12
First_Inst_Purch                      0
TEM_W_IVA                      0.161096
V_Inst                         96645.71
First_Inst_Sold                       0
D_F_Due             2025-01-28 00:00:00
ID_Purch                           None
ID_Sale                            None
Name: 2, dtype: object


In [10]:
df = pd.read_sql('credits', engine, index_col='ID')

df

,ID_External,ID_Client,Date_Settlement,ID_BP,Cap_Requested,Cap_Grant,N_Inst,First_Inst_Purch,TEM_W_IVA,V_Inst,First_Inst_Sold,D_F_Due,ID_Purch,ID_Sale
ID,,,,,,,,,,,,,,
1,None,1,2024-11-28,4,1000000.0,1000000.0,6,0,0.154521,267461.42,0,2025-01-28,None,None
2,None,1,2024-11-28,4,500000.0,500000.0,12,0,0.161096,96645.71,0,2025-01-28,None,None


In [12]:
df = pd.read_sql('installments', engine, index_col='ID')
df = df.loc[df['ID_Op'] == 2]

df.loc['Total', ['Capital', 'Interest', 'IVA', 'Total']] = df[['Capital', 'Interest', 'IVA', 'Total']].sum()

print(f'{df.loc['Total', 'Capital'] + df.loc['Total', 'Interest'] + df.loc['Total', 'IVA'] - df.loc['Total', 'Total']:,.2f}')

df[['Capital', 'Interest', 'IVA', 'Total']] = df[['Capital', 'Interest', 'IVA', 'Total']].map('${:,.2f}'.format)
df

-0.02


,ID_Op,Nro_Inst,D_Due,Capital,Interest,IVA,Total,ID_Owner
ID,,,,,,,,
7,2.0,1.0,2025-01-28,"$16,097.76","$66,568.55","$13,979.40","$96,645.71",1.0
8,2.0,2.0,2025-01-28,"$18,691.05","$64,425.34","$13,529.32","$96,645.71",1.0
9,2.0,3.0,2025-02-28,"$21,702.10","$61,936.87","$13,006.74","$96,645.71",1.0
10,2.0,4.0,2025-03-28,"$25,198.22","$59,047.51","$12,399.98","$96,645.71",1.0
11,2.0,5.0,2025-04-28,"$29,257.54","$55,692.70","$11,695.47","$96,645.71",1.0
12,2.0,6.0,2025-05-28,"$33,970.81","$51,797.43","$10,877.46","$96,645.71",1.0
13,2.0,7.0,2025-06-28,"$39,443.37","$47,274.66","$9,927.68","$96,645.71",1.0
14,2.0,8.0,2025-07-28,"$45,797.54","$42,023.28","$8,824.89","$96,645.71",1.0
15,2.0,9.0,2025-08-28,"$53,175.33","$35,925.93","$7,544.45","$96,645.71",1.0
